# DataLab Task 5: Model Iterations (RNN)

### Summary of the Notebook

This notebook focuses on building and evaluating a Recurrent Neural Network (RNN) model for emotion classification based on extracted NLP features. The workflow includes:

1. **Data Preparation**: Loading and preprocessing features such as TF-IDF, pretrained embeddings, custom embeddings, and other numerical features. The target labels are encoded for classification.
2. **Feature Engineering**: Combining all features into a single dataset and normalizing them for model training.
3. **Model Development**: Constructing an RNN model with multiple layers, including SimpleRNN, BatchNormalization, Dropout, and Dense layers.
4. **Training and Evaluation**: Training the model with callbacks for early stopping, model checkpointing, and F1-score monitoring. The model's performance is evaluated using metrics like accuracy, F1-score, confusion matrix, and classification report.

### Observations
The model exhibits signs of overfitting, as indicated by a significant gap between training and validation performance. This suggests that the model is learning the training data well but struggles to generalize to unseen data. Potential remedies include reducing model complexity, increasing dropout rates, or using regularization techniques.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
from IPython.display import display

# Load the dataset with extracted features
features = "NLP_features.xlsx"
df = pd.read_excel(features)

# Display dataset structure in table format
display(df.head())

,Sentence,POS_Tags,TF_IDF,Sentiment_Score,Pretrained_Embeddings,Custom_Embeddings,Sentiment_Exclamations_Questions,Personal_Pronoun_Count
0,Vous êtes embrassés?,Vous_PRON êtes_AUX embrassés_VERB ?_PUNCT,[0. 0. 0. ... 0. 0. 0.],0.0000,[ 0.044216 -0.0278645 -0.032453 -0.030573...,[ 5.27364027e-04 5.90693962e-04 3.06792255e-...,"0.0,0,1",1
1,Oui.,Oui_ADV ._PUNCT,[0. 0. 0. ... 0. 0. 0.],0.0100,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-3.2684386e-03 4.5674204e-04 -2.1957180e-03 ...,"0.0,0,0",0
2,Mais non!,Mais_CCONJ non_ADV !_PUNCT,[0. 0. 0. ... 0. 0. 0.],-0.0125,[ 1.6874e-01 6.2667e-03 -7.5556e-02 -8.9906e-...,[ 1.61075848e-03 3.01836710e-03 2.69862730e-...,"0.0,1,0",0
3,Vous êtes embrassés?,Vous_PRON êtes_AUX embrassés_VERB ?_PUNCT,[0. 0. 0. ... 0. 0. 0.],0.0000,[ 0.044216 -0.0278645 -0.032453 -0.030573...,[ 5.27364027e-04 5.90693962e-04 3.06792255e-...,"0.0,0,1",1
4,Oui.,Oui_ADV ._PUNCT,[0. 0. 0. ... 0. 0. 0.],0.0100,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-3.2684386e-03 4.5674204e-04 -2.1957180e-03 ...,"0.0,0,0",0


In [3]:
# Convert TF-IDF features
df["TF_IDF"] = df["TF_IDF"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" ") if isinstance(x, str) else x)
tfidf_features = np.array(df["TF_IDF"].tolist())
if len(tfidf_features.shape) == 1:
    tfidf_features = tfidf_features.reshape(-1, 1)

C:\Users\luisf\AppData\Local\Temp\ipykernel_31364\2270948474.py:2: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  df["TF_IDF"] = df["TF_IDF"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" ") if isinstance(x, str) else x)


In [4]:
# Convert embeddings
df["Pretrained_Embeddings"] = df["Pretrained_Embeddings"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" ") if isinstance(x, str) else x)
df["Custom_Embeddings"] = df["Custom_Embeddings"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" ") if isinstance(x, str) else x)
pretrained_embeddings = np.array(df["Pretrained_Embeddings"].tolist())
custom_embeddings = np.array(df["Custom_Embeddings"].tolist())
if len(pretrained_embeddings.shape) == 1:
    pretrained_embeddings = pretrained_embeddings.reshape(-1, 1)
if len(custom_embeddings.shape) == 1:
    custom_embeddings = custom_embeddings.reshape(-1, 1)

# Convert other numerical features
df["Sentiment_Score"] = df["Sentiment_Score"].astype(float)
df["Personal_Pronoun_Count"] = df["Personal_Pronoun_Count"].astype(float)
other_features = df[["Sentiment_Score", "Personal_Pronoun_Count"]].values


In [5]:
# Encode target labels
label_encoder = LabelEncoder()
df["Emotion_Label"] = label_encoder.fit_transform(df["Sentiment_Exclamations_Questions"])

# Combine all features
X = np.hstack((tfidf_features, pretrained_embeddings, custom_embeddings, other_features))
y = df["Emotion_Label"].astype(int).values

In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Normalize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Reshape data for RNN
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Convert labels to categorical
num_classes = len(np.unique(y))
y_train_categorical = to_categorical(y_train, num_classes=num_classes)
y_test_categorical = to_categorical(y_test, num_classes=num_classes)

In [9]:
# Build RNN model
model = Sequential([
    SimpleRNN(128, return_sequences=True, input_shape=(1, X_train_scaled.shape[1])),
    BatchNormalization(),
    Dropout(0.3),
    SimpleRNN(64, return_sequences=False),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

c:\Users\luisf\anaconda3\envs\blockc_y2\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
# Define F1-score callback
class F1ScoreCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        y_pred = np.argmax(self.model.predict(X_test_reshaped), axis=1)
        f1 = f1_score(y_test, y_pred, average='weighted')
        print(f' - F1 Score: {f1:.4f}')

# Set callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_rnn_model.h5', save_best_only=True),
    F1ScoreCallback()
]

In [11]:
# Train the model
history = model.fit(
    X_train_reshaped, y_train_categorical,
    validation_data=(X_test_reshaped, y_test_categorical),
    epochs=50,
    batch_size=32,
    callbacks=callbacks
)

Epoch 1/50
16/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3593 - loss: 2.0894

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
 - F1 Score: 0.8778
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.4114 - loss: 1.9761 - val_accuracy: 0.9018 - val_loss: 1.1878
Epoch 2/50
17/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8594 - loss: 0.8765 

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
 - F1 Score: 0.8878
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8647 - loss: 0.8497 - val_accuracy: 0.9202 - val_loss: 0.7133
Epoch 3/50
16/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8950 - loss: 0.5318 

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
 - F1 Score: 0.9098
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9017 - loss: 0.5103 - val_accuracy: 0.9325 - val_loss: 0.5465
Epoch 4/50
15/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9216 - loss: 0.3944 

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
 - F1 Score: 0.8980
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9275 - loss: 0.3714 - val_accuracy: 0.9202 - val_loss: 0.4805
Epoch 5/50
19/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9637 - loss: 0.2149 

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
 - F1 Score: 0.8924
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9615 - loss: 0.2220 - val_accuracy: 0.9141 - val_loss: 0.4496
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step p - accuracy: 0.9497 - loss: 0.2471
 - F1 Score: 0.8924
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9503 - loss: 0.2428 - val_accuracy: 0.9141 - val_loss: 0.4507
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step p - accuracy: 0.9644 - loss: 0.1883
 - F1 Score: 0.8999
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9635 - loss: 0.1902 - val_accuracy: 0.9202 - val_loss: 0.4503
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step p - accuracy: 0.9405 - loss: 0.2227
 - F1 Score: 0.8999
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9452 - loss: 0.2103 - val_accuracy: 0.9202 - val_loss: 0.4771
Epoch 9/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step p - accuracy: 0.9605 - loss: 0.1400
 - F1 Score: 0.8980
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9617 - l

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
 - F1 Score: 0.8999
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9576 - loss: 0.1555 - val_accuracy: 0.9202 - val_loss: 0.4431
Epoch 11/50
19/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9728 - loss: 0.1023 

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
 - F1 Score: 0.8999
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9727 - loss: 0.1040 - val_accuracy: 0.9202 - val_loss: 0.4401
Epoch 12/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step p - accuracy: 0.9657 - loss: 0.1176
 - F1 Score: 0.8999
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9660 - loss: 0.1178 - val_accuracy: 0.9202 - val_loss: 0.4890
Epoch 13/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step p - accuracy: 0.9877 - loss: 0.0866
 - F1 Score: 0.8999
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9843 - loss: 0.0918 - val_accuracy: 0.9202 - val_loss: 0.4857
Epoch 14/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step p - accuracy: 0.9727 - loss: 0.1034
 - F1 Score: 0.9055
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9732 - loss: 0.1045 - val_accuracy: 0.9264 - val_loss: 0.4868
Epoch 15/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step p - accuracy: 0.9857 - loss: 0.0701
 - F1 Score: 0.9055
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9834

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)
print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions, target_names=label_encoder.classes_)
print("\nClassification Report:")
print(class_report)

In [12]:
# Evaluate final model
final_predictions = np.argmax(model.predict(X_test_reshaped), axis=1)
final_f1 = f1_score(y_test, final_predictions, average='weighted')
print(f'Final F1 Score: {final_f1:.4f}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Final F1 Score: 0.8999
